In [ ]:
# https://skymind.ai/wiki/generative-adversarial-network-gan
import keras
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import mnist
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import LeakyReLU, Dropout
from keras.layers import Input,Conv2D, Conv2DTranspose, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.layers import Reshape
from keras.models import Model

In [ ]:
from numpy import expand_dims
from numpy import ones
from numpy import zeros
from numpy.random import rand
from numpy.random import randint

#Source: https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-a-cifar-10-small-object-photographs-from-scratch/ reffered from this article to build this GAN

  The classes are:

    | Label | Description |
    |:-----:|-------------|
    |   0   | airplane    |
    |   1   | automobile  |
    |   2   | bird        |
    |   3   | cat         |
    |   4   | deer        |
    |   5   | dog         |
    |   6   | frog        |
    |   7   | horse       |
    |   8   | ship        |
    |   9   | truck       |

I chose to run cat images

In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np

class DCGAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Define optimizers
        self.generator_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()

    def build_generator(self):
        model = models.Sequential()
        model.add(layers.Dense(4*4*256, use_bias=False, input_shape=(self.latent_dim,)))
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU())

        model.add(layers.Reshape((4, 4, 256)))

        model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False))
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU())

        model.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False))
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU())

        model.add(layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

        return model

    def build_discriminator(self):
        model = models.Sequential()
        model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                         input_shape=self.img_shape))
        model.add(layers.LeakyReLU())
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
        model.add(layers.LeakyReLU())
        model.add(layers.Dropout(0.3))

        model.add(layers.Flatten())
        model.add(layers.Dense(1))

        return model

    def discriminator_loss(self, real_output, fake_output):
        real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
        fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss

    def generator_loss(self, fake_output):
        return tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output)

    @tf.function
    def train_step(self, images):
        noise = tf.random.normal([self.batch_size, self.latent_dim])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(noise, training=True)

            real_output = self.discriminator(images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            gen_loss = self.generator_loss(fake_output)
            disc_loss = self.discriminator_loss(real_output, fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        return gen_loss, disc_loss

    def train(self, dataset, epochs, batch_size, save_interval=50):
        self.batch_size = batch_size

        for epoch in range(epochs):
            gen_loss_total = 0
            disc_loss_total = 0
            num_batches = 0
            for image_batch in dataset:
                gen_loss, disc_loss = self.train_step(image_batch)
                gen_loss_total += gen_loss
                disc_loss_total += disc_loss
                num_batches += 1

            # Calculate average loss for this epoch
            gen_loss_avg = gen_loss_total / num_batches
            disc_loss_avg = disc_loss_total / num_batches

            # Print the progress
            print(f"Epoch {epoch} [D loss: {disc_loss_avg:.4f}] [G loss: {gen_loss_avg:.4f}]")

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = tf.random.normal([r * c, self.latent_dim])
        gen_imgs = self.generator(noise, training=False)

        # Rescale images 0 - 1
        gen_imgs = (gen_imgs + 1) / 2.0

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(f"image1/aero-cifar_{epoch}.png")
        plt.close()

    def load_data(self, class_label=0):
      (train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()

      # Filter images for the specified class
      class_mask = (train_labels == class_label).reshape(-1)
      train_images = train_images[class_mask]

      train_images = train_images.astype('float32')
      train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

      self.batch_size = 32
      return tf.data.Dataset.from_tensor_slices(train_images).shuffle(len(train_images)).batch(self.batch_size)

if __name__ == '__main__':
    gan = DCGAN()
    dataset = gan.load_data(class_label=3)
    gan.train(dataset, epochs=1001, batch_size=32, save_interval=200)

Epoch 0 [D loss: 0.7346] [G loss: 1.3042]
Epoch 1 [D loss: 0.4912] [G loss: 2.7838]
Epoch 2 [D loss: 0.6643] [G loss: 1.8975]
Epoch 3 [D loss: 0.6234] [G loss: 1.8534]
Epoch 4 [D loss: 0.7908] [G loss: 1.6743]
Epoch 5 [D loss: 1.0102] [G loss: 1.3773]
Epoch 6 [D loss: 1.0249] [G loss: 1.4379]
Epoch 7 [D loss: 1.0925] [G loss: 1.0447]
Epoch 8 [D loss: 1.0578] [G loss: 1.0616]
Epoch 9 [D loss: 1.1765] [G loss: 1.0142]
Epoch 10 [D loss: 1.2134] [G loss: 0.9400]
Epoch 11 [D loss: 1.1925] [G loss: 0.9596]
Epoch 12 [D loss: 1.1805] [G loss: 0.8702]
Epoch 13 [D loss: 1.1802] [G loss: 0.9548]
Epoch 14 [D loss: 1.1826] [G loss: 0.9694]
Epoch 15 [D loss: 1.2172] [G loss: 0.9385]
Epoch 16 [D loss: 1.1072] [G loss: 0.9673]
Epoch 17 [D loss: 1.2449] [G loss: 0.9710]
Epoch 18 [D loss: 1.1639] [G loss: 0.9525]
Epoch 19 [D loss: 1.2605] [G loss: 0.9157]
Epoch 20 [D loss: 1.2548] [G loss: 0.8898]
Epoch 21 [D loss: 1.1442] [G loss: 1.0210]
Epoch 22 [D loss: 1.2817] [G loss: 0.9459]
Epoch 23 [D loss: 1.1